In [6]:
# importujeme potrebne kniznice
import json
from rdflib import Dataset, URIRef, RDF

#  vytvorime RDF dataset a priradime mu meno
ds = Dataset()
graph_name = URIRef("http://hetionet_mapping.org")
graph = ds.graph(graph_name)

# nacitame JSON subor 
with open("/Users/simonsnopko/Desktop/Škola/bakalarska praca/hetionet-v1.0.json", "r") as f:
    data = json.load(f)
    
# prechadzame cez jednotlive hrany (edges) v datach
for edge in data["edges"]:
    # ziskame ID zdrojoveho a cieloveho objektu hrany
    source_id = edge["source_id"][1]
    target_id = edge["target_id"][1]

    # vytvorime URI pre zdrojovy objekt na zaklade jeho typu
    if edge["source_id"][0] == "Compound":
        source_uri = URIRef(f"https://bioregistry.io/drugbank:{source_id}")
    elif edge["source_id"][0] == "Disease":
        source_uri = URIRef(f"https://bioregistry.io/{source_id}")
    elif edge["source_id"][0] == "Anatomy":
        source_uri = URIRef(f"https://bioregistry.io/{source_id}")
    elif edge["source_id"][0] == "Gene":
        source_uri = URIRef(f"https://bioregistry.io/ncbigene:{source_id}")
    elif edge["source_id"][0] == "Pharmacologic Class":
        source_uri = URIRef(f"http://purl.bioontology.org/ontology/NDFRT/{source_id}")
    else:
        continue

    # vytvorime URI pre cielovy objekt na zaklade jeho typu
    if edge["target_id"][0] == "Disease":
        target_uri = URIRef(f"https://bioregistry.io/{target_id}")
    elif edge["target_id"][0] == "Compound":
        target_uri = URIRef(f"https://bioregistry.io/drugbank:{target_id}")
    elif edge["target_id"][0] == "Anatomy":
        target_uri = URIRef(f"https://bioregistry.io/{target_id}")
    elif edge["target_id"][0] == "Gene":
        target_uri = URIRef(f"https://bioregistry.io/ncbigene:{target_id}")
    elif edge["target_id"][0] == "Biological Process":
        target_uri = URIRef(f"https://bioregistry.io/{target_id}")
    elif edge["target_id"][0] == "Cellular Component":
        target_uri = URIRef(f"https://bioregistry.io/{target_id}")
    elif edge["target_id"][0] == "Molecular Function":
        target_uri = URIRef(f"https://bioregistry.io/{target_id}")
    elif edge["target_id"][0] == "Side Effect":
        target_uri = URIRef(f"https://bioregistry.io/sider.effect:{target_id}")
    elif edge["target_id"][0] == "Symptom":
        target_uri = URIRef(f"https://bioregistry.io/mesh:{target_id}")
    elif edge["target_id"][0] == "Pathway" and edge["target_id"][1].startswith("PC7"):
        target_uri = URIRef(f"http://www.wikipathways.org/instance/{target_id}")
    elif edge["target_id"][0] == "Pathway" and edge["target_id"][1].startswith("WP"):
        target_uri = URIRef(f"https://bioregistry.io/wikipathways:{target_id.split('_')[0]}")
    else:
        continue

    # na zaklade typu hrany pridavame triplety do RDF datasetu. Ako predikat urcujeme typ hrany
    # pre hrany ktore maju rovnaky nazov ale odlisny zdrojovy alebo cielovy uzol pridame aj typ relacie pomocou RDF.type
    if edge["kind"] == "treats":
        ds.add((source_uri, URIRef("https://het.io/treats"), target_uri, graph_name))
        
    elif edge["kind"] == "palliates":
        ds.add((source_uri, URIRef("https://het.io/palliates"), target_uri, graph_name))
        
    elif edge["kind"] == "resembles" and edge["source_id"][0] == "Disease":
        ds.add((source_uri, URIRef("https://het.io/resembles"), target_uri, graph_name))
        ds.add((source_uri, RDF.type, URIRef("https://het.io/disease"), graph_name)) 
        
    elif edge["kind"] == "resembles" and edge["source_id"][0] == "Compound":
        ds.add((source_uri, URIRef("https://het.io/resembles"), target_uri, graph_name))
        ds.add((source_uri, RDF.type, URIRef("https://het.io/compound"), graph_name))
        
    elif edge["kind"] == "localizes":
        ds.add((source_uri, URIRef("https://het.io/localizes"), target_uri, graph_name))
             
    elif edge["kind"] == "upregulates" and edge["source_id"][0] == "Compound":
        ds.add((source_uri, URIRef("https://het.io/upregulates"), target_uri, graph_name))
        ds.add((source_uri, RDF.type, URIRef("https://het.io/compound"), graph_name))
        
    elif edge["kind"] == "binds":
        ds.add((source_uri, URIRef("https://het.io/binds"), target_uri, graph_name))
        
    elif edge["kind"] == "downregulates" and edge["source_id"][0] == "Compound":
        ds.add((source_uri, URIRef("https://het.io/downregulates"), target_uri, graph_name))
        ds.add((source_uri, RDF.type, URIRef("https://het.io/compound"), graph_name))
        
    elif edge["kind"] == "downregulates" and edge["source_id"][0] == "Disease":
        ds.add((source_uri, URIRef("https://het.io/downregulates"), target_uri, graph_name))
        ds.add((source_uri, RDF.type, URIRef("https://het.io/disease"), graph_name))
        
    elif edge["kind"] == "upregulates" and edge["source_id"][0] == "Disease":
        ds.add((source_uri, URIRef("https://het.io/upregulates"), target_uri, graph_name))
        ds.add((source_uri, RDF.type, URIRef("https://het.io/disease"), graph_name))
        
    elif edge["kind"] == "associates":
        ds.add((source_uri, URIRef("https://het.io/associates"), target_uri, graph_name))
        
    elif edge["kind"] == "upregulates" and edge["source_id"][0] == "Anatomy":
        ds.add((source_uri, URIRef("https://het.io/upregulates"), target_uri, graph_name))
        ds.add((source_uri, RDF.type, URIRef("https://het.io/anatomy"), graph_name))
        
    elif edge["kind"] == "downregulates" and edge["source_id"][0] == "Anatomy":
        ds.add((source_uri, URIRef("https://het.io/downregulates"), target_uri, graph_name))
        ds.add((source_uri, RDF.type, URIRef("https://het.io/anatomy"), graph_name))
        
    elif edge["kind"] == "expresses":
        ds.add((source_uri, URIRef("https://het.io/expresses"), target_uri, graph_name))
        
    elif edge["kind"] == "regulates":
        ds.add((source_uri, URIRef("https://het.io/regulates"), target_uri, graph_name))
        
    elif edge["kind"] == "interacts":
        ds.add((source_uri, URIRef("https://het.io/interacts"), target_uri, graph_name))
        
    elif edge["kind"] == "covaries":
        ds.add((source_uri, URIRef("https://het.io/covaries"), target_uri, graph_name))
        
    elif edge["kind"] == "participates" and edge["target_id"][0] == "Biological Process":
        ds.add((source_uri, URIRef("https://het.io/participates"), target_uri, graph_name))
        ds.add((target_uri, RDF.type, URIRef("https://het.io/biological_process"), graph_name))
        
    elif edge["kind"] == "participates" and edge["target_id"][0] == "Cellular Component":
        ds.add((source_uri, URIRef("https://het.io/participates"), target_uri, graph_name))
        ds.add((target_uri, RDF.type, URIRef("https://het.io/cellular_component"), graph_name))
                
    elif edge["kind"] == "participates" and edge["target_id"][0] == "Molecular Function":
        ds.add((source_uri, URIRef("https://het.io/participates"), target_uri, graph_name))
        ds.add((target_uri, RDF.type, URIRef("https://het.io/molecular_function"), graph_name))
                
    elif edge["kind"] == "includes":
        ds.add((source_uri, URIRef("https://het.io/includes"), target_uri, graph_name))
        
    elif edge["kind"] == "causes":
        ds.add((source_uri, URIRef("https://het.io/causes"), target_uri, graph_name))
        
    elif edge["kind"] == "presents":
        ds.add((source_uri, URIRef("https://het.io/presents"), target_uri, graph_name))
        
    elif edge["kind"] == "participates" and edge["target_id"][0] == "Pathway":
        ds.add((source_uri, URIRef("https://het.io/participates"), target_uri, graph_name))
        ds.add((target_uri, RDF.type, URIRef("https://het.io/pathway"), graph_name))
    else:
        continue

# ulozime vytvoreny RDF dataset do suboru vo formate nquads
ds.serialize(destination='mapped.nq', format="nquads")

<Graph identifier=N9fc323364a09469ea1da6dfab3b71904 (<class 'rdflib.graph.Dataset'>)>